In [277]:
import requests
import scrapy
import json
REQ_HEADERS={'user-agent': 'UNC MEJO 583'}
resp = requests.get("http://goheels.com/roster.aspx?path=mbball", headers=REQ_HEADERS)
sel = scrapy.Selector(text=resp.text)

In [278]:
def loadPlayer(player_id):
    res = requests.get("http://goheels.com/roster.aspx?rp_id="+player_id, headers=REQ_HEADERS)
    page = scrapy.Selector(text=res.text)
    info = page.css('.sidearm-roster-player-fields')
    headers = info.css('dt').xpath('string()').extract()
    data = info.css('dd').xpath('string()').extract()
    bio = dict(zip(headers, data))
    names = page.css('.sidearm-roster-player-name span::text').extract()
    bio['First Name'] = names[0]
    bio['Last Name'] = names[1]
    bio['Full Name'] = ' '.join(names)
    bio['Number'] = page.css('.sidearm-roster-player-jersey-number::text').extract_first()
    bio['Image'] = page.css('.sidearm-roster-player-image img::attr(src)').extract_first()
    bio['Bio'] = page.css('#sidearm-roster-player-bio .row').xpath('string()').extract_first()
    return bio


In [279]:
stats_key = { 'current_stats': 'Opponent', 'career_stats': 'Season', 'gamehigh_stats': 'Statistic' }
def loadStats(player_id):
    stats_response = requests.get("http://goheels.com/services/responsive-roster-bio.ashx?type=stats&path=mbball&year=2017&rp_id="+player_id, headers=REQ_HEADERS).json()
    stats = {}
    for table_name in stats_response.keys():
        # Skip if no stats table available
        if not stats_response[table_name]: continue
        
        table_stats = {}
        table = scrapy.Selector(text=stats_response[table_name])
        table_header_rows = table.css('thead tr')

        # Fix bottom table which has two levels of headers
        if len(table_header_rows) > 1:
            first_headers = table_header_rows[0].css('th')
            second_headers = table_header_rows[1].css('th').xpath('string()').extract()
            table_headers = []
            for header in first_headers:
                colspan = header.xpath('@colspan').extract_first()
                if colspan:
                    n = int(colspan)
                    old_header = header.xpath('string()').extract_first()
                    table_headers.extend( [old_header+' '+new_header for new_header in second_headers[:n]] )
                    del second_headers[:n]
                else:
                    table_headers.append( header.xpath('string()').extract_first() )
        else:
            table_headers = table_header_rows[0].css('th').xpath('string()').extract()

        # Fix '%' colums by adding context
        for i, header in enumerate(table_headers):
            if header == '%': 
                table_headers[i] = table_headers[i-1] + ' %'

        for row in table.css('tbody tr, tfoot tr'):
            row_numbers = row.css('td, th').xpath('string()').extract()
            row_stats = dict(zip(table_headers, row_numbers))
            table_stats[ row_stats[ stats_key[table_name] ] ] = row_stats

        stats[ table_name ] = table_stats
        
    return stats

In [280]:
table = sel.css('table')[0]
headers = table.css('th').xpath('string()').extract()
players = []
for href in table.xpath('//tr//td//a/@href').extract():
    if 'roster.aspx' in href:
        player_id = re.search('rp_id=(\d+)', href).group(1)
        bio = loadPlayer(player_id)
        bio['Stats'] = loadStats(player_id)
        players.append(bio)

In [281]:
players[0]

{'Bio': '                            \r\n                            FRESHMAN SEASON (2016-17)\r\n\r\nTied a school record by appearing in all 40 games • Averaged 7.7 minutes per game in the primary relief role behind Joel Berry II at the point • Had 49 assists, 42 turnovers and 21 steals • Had 25 assists and 13 turnovers in ACC regular-season play • Made two steals five times • Was on the floor for five minutes vs. Louisville as UNC out-scored the Cards, 10-2 • Played his bestgame in conference play at Duke when he scored four points and had a team-high four assists in eight minutes • Played a seven and a half minute stretch in the first half as UNC cut Duke’s lead from three to one • Was the first time he led UNC in assists since Long Beach State on 11/15 • Had three assists vs. Virginia Tech • Two assists in seven minutes at Boston College • Six points and four assists in 15 minutes against NC State (most minutes since Radford) • Scored six points against Tennessee, including back-t

In [282]:
with open('heels_mbb.json', 'w') as jsonfile:
    json.dump(players, jsonfile)